In [116]:
import pandas as pd
import numpy as np
import torch
import spacy
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from sklearn.linear_model import SGDClassifier, ElasticNetCV
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC


In [52]:
import warnings
warnings.filterwarnings("ignore")

# Descargar stopwords
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('spanish'))

# Cargar modelo de spacy
nlp = spacy.load('es_core_news_sm')

# Función de preprocesamiento
def preprocess_text(text):
    text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ\s]', '', text)  # Eliminar caracteres no alfabéticos
    text = text.lower()  # Convertir a minúsculas
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eslab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
# Cargar y preprocesar datos
df1 = pd.read_csv("hf://datasets/mariagrandury/fake_news_corpus_spanish/test.csv")
df2 = pd.read_csv("https://huggingface.co/datasets/sayalaruano/FakeNewsCorpusSpanish/raw/main/train.csv")
df3 = pd.read_csv('https://drive.google.com/uc?export=download&id=16925KI3IS1_IajpDS65VR8gInzWsQ1ZR')

In [54]:
# Renombrar columnas y procesar texto
for df in [df1, df2, df3]:
    df.rename(columns={"ID": "Id", "CATEGORY": "Category", "TOPICS": "Topic", "SOURCE": "Source", "HEADLINE": "Headline", "TEXT": "Text", "LINK": "Link"}, inplace=True)
    df['Category'].replace({'Fake': False, 'True': True, 'False': False}, inplace=True)
    df['Category'] = df['Category'].map({True: 1, False: 0})
    df['Text'] = df['Text'].apply(preprocess_text)

In [55]:
# Combinar los dataframes
df_combined = pd.concat([df1, df2, df3])

# Limpiar datos faltantes
df_combined['Headline'].fillna(df_combined['Text'].apply(lambda x: ' '.join(x.split()[:10])), inplace=True)
df_combined.dropna(subset=['Source'], inplace=True)
df_combined['Link'].fillna('Unknown', inplace=True)

In [56]:
df_combined.head(3)

,Id,Category,Topic,Source,Headline,Text,Link
0,1,1,Covid-19,El Economista,Covid-19: mentiras que matan,control covid sólo tema médico resto personal ...,https://www.eleconomista.com.mx/opinion/Covid-...
1,2,0,Política,El matinal,El Gobierno podrá acceder a las IPs de los móv...,gobierno pedro sánchez pablo iglesia encontrar...,https://www.elmatinal.com/espana-ultima-hora/e...
2,3,1,Política,El País,La comunidad musulmana catalana denuncia a Vox...,tres federación agrupar mezquita llevar fisc...,https://elpais.com/espana/elecciones-catalanas...


In [57]:
df_google_news = pd.read_csv('data/real_news.csv')
df_concat = pd.read_csv('data/google_scrap.csv')
df_more = pd.read_csv('data/more.csv')
df_news = pd.read_csv('data/news.csv')
df_real_news_google = pd.read_csv('data/real_news_google.csv')
df_real_news = pd.read_csv('data/real_news.csv')


In [58]:
df_concat = pd.concat([df_google_news, df_concat, df_more, df_news, df_real_news_google, df_real_news])

In [59]:
df_concat.value_counts()

query             position  title                                                                                                              body                                                                                                                                              posted           source                    link                                                                                                                                                          
para              36        Andalucía destinará en su presupuesto de 2025 casi 6.700 millones para el \ntejido productivo                      La consejera de Economía, Hacienda y Fondos Europeos de la Junta de \nAndalucía, Carolina España, anuncia que los Presupuestos autonómicos ...    hace 51 minutos  El Economista             https://www.eleconomista.es/economia/noticias/13042723/10/24/andalucia-destinara-en-su-presupuesto-de-2025-casi-6700-millones-para-el-tejido-productivo.html      2
             

In [60]:
df_concat.isnull().sum()

query        0
position     0
title        3
body        79
posted      79
source       3
link         0
dtype: int64

In [61]:
df_concat.duplicated().sum()

308

In [62]:
df_concat = df_concat.drop_duplicates()

In [63]:
df_concat.isnull().sum()

query        0
position     0
title        3
body        71
posted      71
source       3
link         0
dtype: int64

In [64]:
# Eliminar null values
df_concat.dropna(inplace=True)

In [65]:
df_concat.shape

(831, 7)

In [66]:


# Renombrar columnas
df_concat = df_concat.rename(columns={'title': 'Headline', 'body': 'Text', 'link': 'Link'})

# Añadir columna 'Category' con valor 1 en todas las filas
df_concat['Category'] = 1

# Verificar los cambios
df_concat.head()

,query,position,Headline,Text,posted,source,Link,Category
0,para,1,"Muriqi, un regreso perfecto para un Mallorca q...",El 'pirata' volvió al equipo tras un mes de le...,hace 20 minutos,Marca.com,https://www.marca.com/futbol/mallorca/2024/10/...,1
1,para,2,Intervienen en Chiclana una red de grandes dim...,La Policía autonómica ha rescatado a una docen...,hace 30 minutos,Diario de Cádiz,https://www.diariodecadiz.es/chiclana/intervie...,1
2,para,3,Jota Jordi suelta esta bomba en directo para h...,El tertuliano del Chiringuito está tan seguro ...,hace 3 horas,Diario AS,https://as.com/futbol/videos/jota-jordi-suelta...,1
7,para,8,El referéndum en Moldavia para blindar la vía ...,Moldavia denuncia un ataque “sin precedentes” ...,hace 14 horas,EL PAÍS,https://elpais.com/internacional/2024-10-20/el...,1
8,para,9,Adiós al transporte público gratuito para todo...,El descuento en el transporte público finaliza...,hace 5 horas,20Minutos,https://www.20minutos.es/lainformacion/economi...,1


In [67]:
df_concat.shape

(831, 8)

In [68]:


# Asegúrate de que ambas columnas de los datasets tengan los mismos nombres
df_concat = df_concat[['Headline', 'Text', 'Link', 'Category']]  # Seleccionar las columnas que coinciden
df_combined = df_combined[['Id', 'Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']]  # Ya está organizado

# Añadir columnas vacías a df_google_scrap para alinearlo con el segundo dataset
df_concat['Id'] = range(1, len(df_concat) + 1)  # Asignar un Id único a cada fila
df_concat['Topic'] = 'Unknown'  # Añadir columna Topic con un valor predeterminado
df_concat['Source'] = 'Unknown'  # Añadir columna Source con un valor predeterminado

# Ordenar las columnas para que coincidan con el segundo dataset
df_concat = df_concat[['Id', 'Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']]

# Combinar ambos datasets
df_combined = pd.concat([df_combined, df_concat], ignore_index=True)

# Verificar el dataset combinado
df_combined.head()

,Id,Category,Topic,Source,Headline,Text,Link
0,1,1,Covid-19,El Economista,Covid-19: mentiras que matan,control covid sólo tema médico resto personal ...,https://www.eleconomista.com.mx/opinion/Covid-...
1,2,0,Política,El matinal,El Gobierno podrá acceder a las IPs de los móv...,gobierno pedro sánchez pablo iglesia encontrar...,https://www.elmatinal.com/espana-ultima-hora/e...
2,3,1,Política,El País,La comunidad musulmana catalana denuncia a Vox...,tres federación agrupar mezquita llevar fisc...,https://elpais.com/espana/elecciones-catalanas...
3,4,0,Política,AFPFactual,dar conocer dato electoral preliminar persona ...,dar conocer dato electoral preliminar persona ...,https://perma.cc/GYE6-SPMB
4,5,1,Sociedad,La Republica,El censo poblacional 2018 tendrá un costo de $...,primero fase censo virtual solo abril próximo ...,https://www.larepublica.co/economia/el-censo-p...


In [69]:
df_combined.shape


(2367, 7)

In [70]:
df_combined.isnull().sum()

Id          0
Category    0
Topic       0
Source      0
Headline    0
Text        0
Link        0
dtype: int64

In [71]:
df_combined.duplicated().sum()

0

In [72]:
df_fake_news = pd.read_csv('data/headline_links_combined.csv')

In [73]:
df_fake_news.head()

,Index,Headline,Link
0,0,"“En el AIFA hay más ratas que vuelos”, afirmau...",https://aifa.aero/
1,0,"Afirmaciones similares circulan en Twitter (1,...",https://www.nytimes.com/es/2022/03/25/espanol/...
2,0,El Aeropuerto Internacional Felipe Ángeles (AI...,https://www.aicm.com.mx/
3,0,"Sin embargo, el Felipe Ángeles ha tardado en a...",https://www.aicm.com.mx/acercadelaicm/archivos...
4,0,“La gente sigue sin darse cuenta pero esto es ...,https://www.latribune.fr/entreprises-finance/b...


In [74]:
df_fake_news.shape

(12514, 3)

In [75]:
df_fake_news.isnull().sum()

Index       0
Headline    0
Link        0
dtype: int64

In [76]:
df_fnews = df_fake_news.head(1000)

In [77]:
df_fnews = df_fnews.rename(columns={'Headline': 'Text'})
df_fnews['Category'] = 0    

In [78]:
df_fnews.head()

,Index,Text,Link,Category
0,0,"“En el AIFA hay más ratas que vuelos”, afirmau...",https://aifa.aero/,0
1,0,"Afirmaciones similares circulan en Twitter (1,...",https://www.nytimes.com/es/2022/03/25/espanol/...,0
2,0,El Aeropuerto Internacional Felipe Ángeles (AI...,https://www.aicm.com.mx/,0
3,0,"Sin embargo, el Felipe Ángeles ha tardado en a...",https://www.aicm.com.mx/acercadelaicm/archivos...,0
4,0,“La gente sigue sin darse cuenta pero esto es ...,https://www.latribune.fr/entreprises-finance/b...,0


In [79]:
# Obtener las columnas comunes
common_columns = df_combined.columns.intersection(df_fnews.columns)

In [80]:
common_columns

Index(['Category', 'Text', 'Link'], dtype='object')

In [81]:
df_combined_filtered = df_combined[common_columns]

In [82]:
# Concatenar ambos DataFrames usando las columnas comunes
df_combined = pd.concat([df_combined, df_fnews], ignore_index=True)

# Ver las primeras filas del nuevo DataFrame unido
df_combined.head()

,Id,Category,Topic,Source,Headline,Text,Link,Index
0,1.0,1,Covid-19,El Economista,Covid-19: mentiras que matan,control covid sólo tema médico resto personal ...,https://www.eleconomista.com.mx/opinion/Covid-...,NaN
1,2.0,0,Política,El matinal,El Gobierno podrá acceder a las IPs de los móv...,gobierno pedro sánchez pablo iglesia encontrar...,https://www.elmatinal.com/espana-ultima-hora/e...,NaN
2,3.0,1,Política,El País,La comunidad musulmana catalana denuncia a Vox...,tres federación agrupar mezquita llevar fisc...,https://elpais.com/espana/elecciones-catalanas...,NaN
3,4.0,0,Política,AFPFactual,dar conocer dato electoral preliminar persona ...,dar conocer dato electoral preliminar persona ...,https://perma.cc/GYE6-SPMB,NaN
4,5.0,1,Sociedad,La Republica,El censo poblacional 2018 tendrá un costo de $...,primero fase censo virtual solo abril próximo ...,https://www.larepublica.co/economia/el-censo-p...,NaN


In [83]:
df_combined.shape

(3367, 8)

In [84]:
df_combined.isnull().sum()

Id          1000
Category       0
Topic       1000
Source      1000
Headline    1000
Text           0
Link           0
Index       2367
dtype: int64

In [85]:
columns_to_drop = ['Topic', 'Source', 'Headline','Index', 'Id']  # Cambia los nombres de las columnas según sea necesario

# Eliminar las columnas específicas
df_combined = df_combined.drop(columns=columns_to_drop)



In [86]:
# Extraer el nombre del dominio
df_combined['Source'] = df_combined['Link'].str.extract(r'https?://(?:www\.)?([^/]+)')


In [92]:
df_combined['Source'] = df_combined['Source'].str.split('.').str[0]

In [93]:
# Mostrar el DataFrame limpio
display(df_combined)

,Category,Text,Link,Source
0,1,control covid sólo tema médico resto personal ...,https://www.eleconomista.com.mx/opinion/Covid-...,eleconomista
1,0,gobierno pedro sánchez pablo iglesia encontrar...,https://www.elmatinal.com/espana-ultima-hora/e...,elmatinal
2,1,tres federación agrupar mezquita llevar fisc...,https://elpais.com/espana/elecciones-catalanas...,elpais
3,0,dar conocer dato electoral preliminar persona ...,https://perma.cc/GYE6-SPMB,perma
4,1,primero fase censo virtual solo abril próximo ...,https://www.larepublica.co/economia/el-censo-p...,larepublica
...,...,...,...,...
3362,0,"Nos habéis preguntado por WhatsApp, Twitter e ...",https://www.osi.es/es/actualidad/avisos/2020/0...,osi
3363,0,En la parte superior del falso artículo se pue...,https://maldita.es/malditobulo/2020/05/12/sms-...,maldita
3364,0,"Si se pega ese enlace en el navegador,sale un ...",https://www.mercadona.es/,mercadona
3365,0,En ningún lugar de ese artículo se habla de Sa...,https://maldita.es/malditobulo/2020/04/06/merc...,maldita


In [94]:
df_combined.head(5)

,Category,Text,Link,Source
0,1,control covid sólo tema médico resto personal ...,https://www.eleconomista.com.mx/opinion/Covid-...,eleconomista
1,0,gobierno pedro sánchez pablo iglesia encontrar...,https://www.elmatinal.com/espana-ultima-hora/e...,elmatinal
2,1,tres federación agrupar mezquita llevar fisc...,https://elpais.com/espana/elecciones-catalanas...,elpais
3,0,dar conocer dato electoral preliminar persona ...,https://perma.cc/GYE6-SPMB,perma
4,1,primero fase censo virtual solo abril próximo ...,https://www.larepublica.co/economia/el-censo-p...,larepublica


In [97]:
min_samples_threshold = 5
source_counts = df_combined['Source'].value_counts()
sources_to_keep = source_counts[source_counts >= min_samples_threshold].index

# Agrupar fuentes raras en "Otros"
df_combined['Source'] = df_combined['Source'].apply(lambda x: x if x in sources_to_keep else "Otros")

In [112]:
df_combined['Source'].value_counts()

Source
Otros                728
maldita              335
elpais               213
eldizque             143
osi                  123
                    ... 
20minutos              5
telecinco              5
laopiniondemalaga      5
informacion            5
farodevigo             5
Name: count, Length: 96, dtype: int64

In [113]:
df_combined.shape

(3367, 4)

In [98]:
# División de datos de entrenamiento y prueba
X = df_combined['Text']
y = df_combined['Source']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [99]:
# Vectorización TF-IDF
# Aumentar el número de características en el vectorizador TF-IDF
# Convertir el conjunto de stopwords a una lista
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3), stop_words=list(stop_words))

# Transformar los datos de entrenamiento y prueba
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [100]:
# Filtrado de clases pequeñas
unique_classes, counts = np.unique(y_train, return_counts=True)
class_distribution = pd.DataFrame({'Class': unique_classes, 'Count': counts})
min_samples_threshold = 3
classes_to_keep = class_distribution[class_distribution['Count'] >= min_samples_threshold]['Class'].tolist()

y_train_filtered = y_train[y_train.isin(classes_to_keep)]
X_train_filtered = X_train_vec[y_train.isin(classes_to_keep)]

In [101]:
# Resampling con SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_filtered, y_train_filtered)

In [107]:
# Ajuste de hiperparámetros con GridSearchCV para RandomForest
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(X_train_resampled, y_train_resampled)


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]})

In [111]:
grid_search.best_params_

{'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}

In [108]:
# Clasificadores: Logistic Regression, Ridge, Random Forest, SVM
classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Ridge Classifier": RidgeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC()
}

In [ ]:
classifiers = {


    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "SGD Classifier": SGDClassifier(),
    "ElasticNet": ElasticNetCV(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": MultinomialNB(),
    "MLP Classifier": MLPClassifier(max_iter=500),
    "Linear SVC": LinearSVC(),
    "Gaussian Naive Bayes": GaussianNB(),
}

In [109]:
# Entrenar los modelos y generar predicciones
predictions = {}
for name, clf in classifiers.items():
    clf.fit(X_train_resampled, y_train_resampled)
    predictions[name] = clf.predict(X_test_vec)

In [110]:
# Reportes de clasificación para cada modelo
for name, preds in predictions.items():
    print(f"\n{name} Report:")
    print(classification_report(y_test, preds))


Logistic Regression Report:
                     precision    recall  f1-score   support

          20minutos       0.00      0.00      0.00         0
              Otros       0.47      0.50      0.48       107
                abc       0.36      0.44      0.40         9
      alertadigital       0.00      0.00      0.00         3
                api       0.00      0.00      0.00         2
              apisa       0.00      0.00      0.00         2
  argumentopolitico       0.00      0.00      0.00         2
  aristeguinoticias       0.00      0.00      0.00         2
                 as       1.00      0.50      0.67         2
        atresplayer       0.00      0.00      0.00         4
                bbc       0.22      0.44      0.30         9
          cadenaser       0.00      0.00      0.00         3
            caritas       0.00      0.00      0.00         3
           censura0       0.30      0.50      0.38         6
          cincodias       0.00      0.00      0.00     